**MLFlow**

In [2]:
!nvidia-smi

Sun May  4 17:17:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   45C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import mlflow
import mlflow.pytorch

# --- Model Definition ---
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        return self.fc2(x)

# --- Train Function ---
def train(model, device, train_loader, optimizer, epoch, criterion):
    model.train()
    total_loss = 0
    correct = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    return total_loss / len(train_loader), correct / len(train_loader.dataset)

# --- Main MLflow Logging ---
def main():
    # --- Setup ---
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.ToTensor()
    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    model = SimpleNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # --- MLflow Tracking ---
    with mlflow.start_run():
        mlflow.log_param("epochs", 5)
        mlflow.log_param("batch_size", 64)
        mlflow.log_param("lr", 0.001)

        for epoch in range(1, 6):
            loss, acc = train(model, device, train_loader, optimizer, epoch, criterion)
            print(f"Epoch {epoch}: Loss={loss:.4f}, Accuracy={acc:.4f}")
            mlflow.log_metric("loss", loss, step=epoch)
            mlflow.log_metric("accuracy", acc, step=epoch)

        # Log the model
        mlflow.pytorch.log_model(model, "model")

if __name__ == "__main__":
    main()

Epoch 1: Loss=0.3474, Accuracy=0.9042
Epoch 2: Loss=0.1576, Accuracy=0.9534
Epoch 3: Loss=0.1106, Accuracy=0.9674
Epoch 4: Loss=0.0846, Accuracy=0.9750


2025/05/03 22:48:03 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Epoch 5: Loss=0.0659, Accuracy=0.9800


2025/05/03 22:48:09 WARNING mlflow.utils.requirements_utils: Found torch version (2.6.0+cu124) contains a local version label (+cu124). MLflow logged a pip requirement for this package as 'torch==2.6.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/05/03 22:48:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import mlflow
import mlflow.pytorch

from xlstm.xlstm import xLSTMBlockStack, xLSTMBlockStackConfig

# Define the xLSTM-based model
class xLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super(xLSTMClassifier, self).__init__()
        config = xLSTMBlockStackConfig(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers
        )
        self.xlstm = xLSTMBlockStack(config)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: [batch_size, 1, 28, 28] -> [batch_size, 28, 28]
        x = x.squeeze(1)
        # x: [batch_size, 28, 28]
        output, _ = self.xlstm(x)
        # Use the output from the last time step
        out = self.fc(output[:, -1, :])
        return out

# Training function
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    total_loss = 0
    correct = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
    avg_loss = total_loss / len(train_loader.dataset)
    accuracy = correct / len(train_loader.dataset)
    return avg_loss, accuracy

# Main function
def main():
    # Hyperparameters
    batch_size = 64
    epochs = 5
    learning_rate = 0.001
    input_dim = 28
    hidden_dim = 128
    num_layers = 2
    num_classes = 10

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data loaders
    transform = transforms.ToTensor()
    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Model, loss, optimizer
    model = xLSTMClassifier(input_dim, hidden_dim, num_layers, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # MLflow tracking
    with mlflow.start_run():
        mlflow.log_params({
            "batch_size": batch_size,
            "epochs": epochs,
            "learning_rate": learning_rate,
            "input_dim": input_dim,
            "hidden_dim": hidden_dim,
            "num_layers": num_layers
        })

        for epoch in range(1, epochs + 1):
            loss, acc = train(model, device, train_loader, optimizer, criterion, epoch)
            print(f"Epoch {epoch}: Loss={loss:.4f}, Accuracy={acc:.4f}")
            mlflow.log_metric("loss", loss, step=epoch)
            mlflow.log_metric("accuracy", acc, step=epoch)

        # Log the model
        mlflow.pytorch.log_model(model, "model")

if __name__ == "__main__":
    main()

In [ ]:
!pip uninstall -y xlstm

Found existing installation: xlstm 2.0.3
Uninstalling xlstm-2.0.3:
  Successfully uninstalled xlstm-2.0.3


In [ ]:
!git clone https://github.com/NX-AI/xlstm.git
%cd xlstm
!pip install -e

Cloning into 'xlstm'...
remote: Enumerating objects: 399, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 399 (delta 58), reused 61 (delta 40), pack-reused 305 (from 1)
Receiving objects: 100% (399/399), 265.27 KiB | 2.53 MiB/s, done.
Resolving deltas: 100% (211/211), done.
/content/xlstm

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-e option requires 1 argument


In [ ]:
pip install torch torchvision mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2